# AgnoAgent 笔记本 — 说明与运行指南

## 一、整体结构（从上到下在做什么）

| 区块 | 作用 |
|------|------|
| **0. 安装** | 安装 agno、openai、google-genai、mcp、fastapi、sqlalchemy |
| **1. 基础 Agent** | 用 OpenRouter + InMemoryDb 建一个只会聊天的 Agent，并跑一句 "Hello" |
| **2. Agent + 工具 + 结构化输出** | 给 Agent 一个「加法」工具，并让回复变成固定格式（chat / answer / tools_used） |
| **3. 面试 Agent（简化版）** | 单 Agent：根据「技术面试 + 数据科学家」上下文，每次只出一道面试题，输出结构化（题目、类型、期望要点） |
| **4. 完整面试系统** | 引入：RAG 知识库、State、多个工具、Coach Agent、Team、Session Summary、Workflow |

## 二、各块在做什么（逐段说明）

### 块 1：安装依赖（第一个代码单元）
- 安装运行本笔记本需要的包：`agno`、`openai`、`google-genai`、`mcp`、`fastapi`、`sqlalchemy`。
- **运行一次即可**，环境已有的话会显示已满足。

### 块 2：基础 Agent（OpenRouter + 记忆）
- 用 **OpenRouter** 接 `gpt-4o-mini`，用 **InMemoryDb** 存对话历史。
- 建一个叫 `BasicAgent` 的 Agent，只会按「You are a helpful AI Assistant」回复。
- **运行后**：下一个单元里 `agent.run("Hello, how are you?")` 就会得到一句问候回复。

### 块 3：Agent + 工具 + 结构化输出
- **工具**：定义一个 `add(a, b)`，Agent 可以调用来做加法。
- **结构化输出**：用 Pydantic 定义 `Response`，包含 `chat`（文字回复）、`answer`（数值答案）、`tools_used`（用了哪些工具）。
- 问 "What's 10 + 2?" 时，Agent 会调用 `add`，并把结果填进 `Response`。
- 这里演示：**Tool + Structured Output** 两个概念。

### 块 4：面试 Agent（第一阶段，单 Agent）
- **上下文**：面试类型 = 技术面试，角色 = 数据科学家（写在 instructions 里）。
- **结构化输出**：`InterviewTurn` = 当前题目、题目类型、期望要点。
- 没有 RAG/State/Team，只是「每次只出一道题、输出格式固定」的面试官。
- 运行 `interview_agent.run("I'm ready. Please give me the first question.")` 会得到第一道题和期望要点。

### 块 5：完整版 — RAG 知识库
- 从本地文件 **ds notes.markdown** 按 `##` 标题拆成多个「章节」。
- 用 **ChromaDb** 建向量库，把每个章节塞进 **Knowledge**，供后面「按主题出题」用。
- **注意**：`DS_NOTES_PATH` 要改成你本机的 `ds notes.markdown` 路径；没有的话要自己准备该文件或改路径。

### 块 6：State + 工具（面试用）
- **State**：记录 `used_section_titles`（已用过的章节）、`round`、`session_start_time`、`interview_ended`。
- **工具**：
  - `get_current_time`：当前时间；
  - `pick_topic_from_rag`：从 RAG 里选一个**还没用过**的主题，用来出题；
  - `end_interview`：标记面试结束。
- 这些工具在后面的 Interviewer Agent 里会被调用。

### 块 7：Coach Agent + request_feedback
- **Coach Agent**：专门做「反馈/评分」，输入题目 + 候选人答案 + RAG 参考，输出分数、覆盖要点、遗漏要点、建议。
- **request_feedback**：一个工具，Interviewer 在候选人答完题后调用，内部会查 RAG、调 Coach，把结构化反馈转成一段文字给用户。

### 块 8：Interviewer Agent + Team
- **Interviewer Agent**：负责出题、控场，带着上面所有工具 + State + RAG，输出 `InterviewTurn`（题目、类型、期望要点，结束时可有 session_summary）。
- **Team**：把 Interviewer 和 Coach 放在一个 Team 里；实际流程里 Coach 是通过 `request_feedback` 被 Interviewer 调用的，不是 Team 自动路由。

### 块 9：Session Summary + Workflow
- **Session Summary**：面试结束时（例如用户说「结束」），用 `generate_session_summary(agent)` 再跑一次 Agent，根据对话历史生成几句总结。
- **Workflow**：  
  - 先「设定上下文」→ 多轮问答（你答一题，Agent 出下一题或给反馈）→ 用户说结束 → 调用 `end_interview` → 再生成 Session Summary。

### 块 10：怎么「跑」一遍
- **单轮演示**：运行「要第一题」的单元 → 再运行「模拟候选人回答 + 请求反馈」→ 再运行「结束面试 + 生成 Session Summary」。
- **交互式多轮**：取消注释最后的 `run_interview_workflow(interviewer_agent, max_rounds=5)`，在终端里按轮次输入你的回答，说「结束」或「end」会结束并打印总结。

## 三、如何运行（推荐顺序）

1. **环境**：确保已安装 Python，建议用 notebook 对应的环境（如 `agno_env`）。
2. **API Key**：在环境变量里设置 `OPENROUTER_API_KEY`，或在项目目录下建 `.env` 文件写 `OPENROUTER_API_KEY=你的key`（运行时会用 `load_dotenv()` 加载；不要提交 `.env` 或 key 到仓库）。
3. **从顶到底顺序执行**：  
   - 先跑「安装」单元；  
   - 再按顺序跑完「基础 Agent」和「Agent + 工具 + 结构化输出」；  
   - 若要试「完整面试系统」，请先准备好 `ds notes.markdown` 并改好 `DS_NOTES_PATH`，再按顺序执行 RAG、State+工具、Coach、Interviewer、Team、Summary、Workflow 各块。
4. **只试「简化版面试」**：只运行到「面试 Agent（第一阶段）」那一段即可，不需要 RAG/ChromaDb。
5. **交互式面试**：在完整版都跑通后，取消注释并执行 `run_interview_workflow(interviewer_agent, max_rounds=5)`，在输入框里输入你的回答，说「结束」结束面试。

## 四、依赖与路径注意

- **ChromaDb**：完整版需要；若报错可取消注释 `!pip install chromadb` 并执行。
- **OPENROUTER_API_KEY**：必须在环境变量中设置且有效；不要将 key 写在代码或提交到 public 仓库。
- **DS_NOTES_PATH**：完整版 RAG 依赖的 Markdown 路径，需改成你本机实际路径。

In [1]:
!pip install agno openai google-genai mcp fastapi sqlalchemy

# Build Agno Agent with OpenRouter

Build a basic agent, then expand the agent with other Agno modules

Ask me for an OpenRouter API key

- [Agno Docs: Building Agents](https://docs.agno.com/basics/agents/building-agents)
- [Agno Docs: OpenRouter](https://docs.agno.com/integrations/models/gateways/openrouter/overview)
- [Cookbook/Examples](https://github.com/agno-agi/agno/tree/main/cookbook)

Set `OPENROUTER_API_KEY` in your environment, or in the code cells below.

### Models
[See all OpenRouter models](https://openrouter.ai/models)
- Search by price, category, parameters allowed, etc
- **Price**: Use Free models as able, but you might need some of the paid ones for more complex tasks.
    - Try to use ones that cost less than $1.00 per 1M / output tokens

In [1]:
import os
from textwrap import dedent
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from agno.db.in_memory import InMemoryDb

# Load environment variables from .env file if it exists
load_dotenv()

# Use env var OPENROUTER_API_KEY only (do not put API keys in this repo)
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment (e.g. export OPENROUTER_API_KEY=your_key) or in .env file. Do not commit keys.")
DB = InMemoryDb()
SESSION_ID = 'session001'

agent = Agent(
    name="BasicAgent",
    model=OpenRouter(
        id='openai/gpt-4o-mini',
        api_key=os.environ.get("OPENROUTER_API_KEY"),
    ),
    instructions=dedent(
        """
        You are a helpful AI Assistant
        """
    ),
    db=DB,
    session_id=SESSION_ID,
    add_history_to_context=True,
)

In [2]:
response = agent.run("Hello, how are you?")
print(response.content)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


## Tasks
Review all the modules of Agno SDK and build an over-engineered Agent that demonstrates key concepts and features

1. [Agno Context Engineering](https://docs.agno.com/context/engineering/overview)

2. [Structured Outputs with Pydantic](https://docs.agno.com/basics/input-output/overview#structured-output)

2. [State Management / Agentic State](https://docs.agno.com/context/state/overview)

3. [Knowledge / Agentic RAG](https://docs.agno.com/context/knowledge/overview)

4. [Session Summary Agent](https://docs.agno.com/context/knowledge/overview)

5. [Tools and Toolkits](https://docs.agno.com/integrations/toolkits/overview)

6. [Reasoning](https://docs.agno.com/features/reasoning/overview)

7. [Build a second agent and use as a `Team`](https://docs.agno.com/basics/teams/overview)

8. [Build `Steps` and use in a `Workflow`](https://docs.agno.com/basics/workflows/building-workflows)

> Does not need to especially sophisticated, and does not need to be an especially useful agent, just work to get the different elements integrated

For example, add a basic tool and structured outputs to your agent:

### Agent with Tool and Structured Output

In [4]:
import os
from textwrap import dedent
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from agno.db.in_memory import InMemoryDb
from agno.tools import tool
from pydantic import BaseModel, Field
from typing_extensions import Optional, List

# Load environment variables from .env file if it exists
load_dotenv()

# OPENROUTER_API_KEY must be set in environment (do not commit keys)
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment or .env file. Do not commit keys.")
DB = InMemoryDb()
SESSION_ID = 'session001'

# Create simple arithmetic tool
@tool
def add(a: int, b: int) -> int:
    """ A tool that adds integers together """
    return a + b

# Create structured output schema with Typed data
class Response(BaseModel):
    """
    A response to a user input that includes:
    - a normal LLM chat response
    - a numerical answer if the user input required math tools
    - a list of tools used (names of tools)
    """
    chat: str = Field(..., description="The chat response to the user input")
    answer: Optional[int] = Field(
        None,
        description="Numerical answer to the user input if it required math tools"
    )
    tools_used: List[str]

# Create agent
agent = Agent(
    name="BasicAgent",
    model=OpenRouter(
        id='openai/gpt-4o-mini',
        api_key=os.environ.get('OPENROUTER_API_KEY'),
    ),
    instructions=dedent(
        """
        You are a helpful AI Assistant. You can add two numbers together
        using a tool, which is more accurate than LLM native arithmetic.
        """
    ),
    tools=[add],
    db=DB,
    session_id=SESSION_ID,
    add_history_to_context=True,
    output_schema=Response,
    use_json_mode=True,  # Helps with some endpoint/model compatibility issues
    retries=3
)

# We'll skip the pretty print response and get the data as a dict/json
run = agent.run("Hey! What's 10 + 2?")  # to display output
print(run.content.model_dump())

# Now we can access response items programatically, enabling all sorts of follow on operations
print("\nAccess individual fields from response:")
print(f"{type(run.content.chat)}: {run.content.chat}")
print(f"{type(run.content.answer)}: {run.content.answer}")
print(f"{type(run.content.tools_used)}: {run.content.tools_used}")

{'chat': 'The sum of 10 and 2 is 12.', 'answer': 12, 'tools_used': ['functions.add']}

Access individual fields from response:
<class 'str'>: The sum of 10 and 2 is 12.
<class 'int'>: 12
<class 'list'>: ['functions.add']


In [7]:
from agno.agent import Agent
import inspect

print(inspect.getsource(Agent.run))

    def run(
        self,
        input: Union[str, List, Dict, Message, BaseModel, List[Message]],
        *,
        stream: Optional[bool] = None,
        stream_events: Optional[bool] = None,
        user_id: Optional[str] = None,
        session_id: Optional[str] = None,
        session_state: Optional[Dict[str, Any]] = None,
        run_context: Optional[RunContext] = None,
        run_id: Optional[str] = None,
        audio: Optional[Sequence[Audio]] = None,
        images: Optional[Sequence[Image]] = None,
        videos: Optional[Sequence[Video]] = None,
        files: Optional[Sequence[File]] = None,
        knowledge_filters: Optional[Union[Dict[str, Any], List[FilterExpr]]] = None,
        add_history_to_context: Optional[bool] = None,
        add_dependencies_to_context: Optional[bool] = None,
        add_session_state_to_context: Optional[bool] = None,
        dependencies: Optional[Dict[str, Any]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        output

### Interview Agent (Phase 1: Single Agent + Structured Output + Context)

Mock interview coach: one agent asks questions based on **context** (interview type, role) and returns **structured output** (current question + type + expected key points). No RAG/State/Team yet—just get the conversation working.

In [8]:
import os
from textwrap import dedent
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from agno.db.in_memory import InMemoryDb
from pydantic import BaseModel, Field
from typing import List

# Load environment variables from .env file if it exists
load_dotenv()

# OPENROUTER_API_KEY must be set in environment
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment or .env file. Do not commit keys.")
DB = InMemoryDb()
SESSION_ID = "interview_session_001"

# --- Context (simple context engineering) ---
INTERVIEW_TYPE = "technical"
ROLE = "Data Scientist"

# --- Structured output: one interview turn = question + type + expected key points ---
class InterviewTurn(BaseModel):
    """One interviewer turn: the question to ask and what to look for in the answer."""
    current_question: str = Field(..., description="The interview question to ask the candidate")
    question_type: str = Field(..., description="e.g. technical, behavioral, system_design")
    expected_key_points: List[str] = Field(..., description="Key points or themes to look for in a good answer")

# --- Interview Agent: single agent, structured output, context in instructions ---
interview_agent = Agent(
    name="InterviewAgent",
    model=OpenRouter(
        id="openai/gpt-4o-mini",
        api_key=os.environ.get("OPENROUTER_API_KEY"),
    ),
    instructions=dedent(f"""
        You are a professional interviewer conducting a mock data science interview.
        Interview type: {INTERVIEW_TYPE}.
        Role: {ROLE}.
        Focus on data science topics: statistics, ML, Python/pandas, SQL, A/B testing, metrics, etc.
        Ask exactly ONE question per turn. Be concise and professional.
        Output your question and the expected key points in the required structured format.
        """),
    db=DB,
    session_id=SESSION_ID,
    add_history_to_context=True,
    output_schema=InterviewTurn,
    use_json_mode=True,
    retries=3,
)

# --- Run: ask for the first question ---
run = interview_agent.run("I'm ready. Please give me the first question.")
print("Raw output (dict):", run.content.model_dump())
print("\n--- Formatted ---")
print(f"Question: {run.content.current_question}")
print(f"Type: {run.content.question_type}")
print(f"Expected key points: {run.content.expected_key_points}")

Raw output (dict): {'current_question': 'Can you explain the difference between Type I and Type II errors in hypothesis testing?', 'question_type': 'technical', 'expected_key_points': ['Type I error occurs when the null hypothesis is rejected when it is actually true (false positive).', 'Type II error occurs when the null hypothesis is not rejected when it is actually false (false negative).', 'The significance level (alpha) is associated with Type I error probability.', 'The power of a test is related to Type II error probability (1 - beta).', 'Importance of balancing both types of errors based on the context of the hypothesis test.']}

--- Formatted ---
Question: Can you explain the difference between Type I and Type II errors in hypothesis testing?
Type: technical
Expected key points: ['Type I error occurs when the null hypothesis is rejected when it is actually true (false positive).', 'Type II error occurs when the null hypothesis is not rejected when it is actually false (false n

---
## Interview Agent (Full): State + RAG + Tools + Team + Session Summary + Workflow

- **State**: Track `used_section_titles`, `round`, `session_start_time`, `interview_ended`
- **RAG**: DS notes markdown chunked by `##`, used for question generation and feedback reference
- **Tools**: `get_current_time`, `pick_topic_from_rag`, `end_interview`, `request_feedback` (calls Coach)
- **Team**: Interviewer (出题/控场) + Coach (反馈/评分)
- **Session Summary**: Triggered on "end interview", included in structured output or displayed separately
- **Workflow**: Set context → multi-round Q&A (team) → summary

In [16]:
# Optional: install chromadb for vector store (run this cell if you get ModuleNotFoundError: chromadb)
!pip install chromadb

  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-macosx_10_12_x86_64.whl.metadata (4.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 34.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 33.3 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 37.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 23.3 MB/s  0:00:00
Using cached hf_xet-1.2.0-cp37-abi3-macosx_10_12_x86_64.whl (2.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.7/748.7 kB 33.7 MB/s  0:00:00
Using cached importlib_resources-6.5.2-py3-none-any.whl (

In [ ]:
# --- 1. RAG: Build knowledge base from DS notes (chunk by ## sections) ---
import os
import re
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
load_dotenv()

# Check if OPENROUTER_API_KEY is set
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment or .env file. Do not commit keys.")

# ChromaDb default embedder may need OPENAI_API_KEY; or pass a custom embedder to ChromaDb
DS_NOTES_PATH = "/Users/joy/Desktop/ds notes.markdown"

def parse_markdown_by_h2(path: str) -> list[tuple[str, str]]:
    """Parse markdown file into (section_title, content) pairs by ## headers."""
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    # Split by ## but keep the header line with the content
    pattern = re.compile(r"^##\s+(.+)$", re.MULTILINE)
    parts = pattern.split(text)
    # parts[0] is content before first ##; then [title1, content1, title2, content2, ...]
    sections = []
    if len(parts) <= 1:
        if text.strip():
            sections.append(("general", text))
        return sections
    # Skip leading content without ## (e.g. first line or intro)
    i = 1
    while i + 1 < len(parts):
        title, content = parts[i].strip(), (parts[i + 1] or "").strip()
        if title or content:
            sections.append((title or "section", content or "(no content)"))
        i += 2
    return sections

sections = parse_markdown_by_h2(DS_NOTES_PATH)
print(f"Parsed {len(sections)} sections (##). First 3 titles: {[t for t, _ in sections[:3]]}")

Parsed 79 sections (##). First 3 titles: ['ML训练中错误来源', '防止过拟合手段', '数据标准化']


In [ ]:
from agno.knowledge import Knowledge
from agno.knowledge.embedder.openai import OpenAIEmbedder
from agno.vectordb.chroma import ChromaDb
from chromadb import PersistentClient
from pathlib import Path
import os

CHROMA_PATH = "./tmp_interview_chroma"
COLLECTION = "ds_notes_emb1536"

Path(CHROMA_PATH).mkdir(parents=True, exist_ok=True)

EMBED_MODEL = "text-embedding-3-small"  # 1536 dim
embedder = OpenAIEmbedder(
    id=EMBED_MODEL,
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

# 1️⃣ 先彻底删旧库
client = PersistentClient(path=CHROMA_PATH)
if COLLECTION in [c.name for c in client.list_collections()]:
    client.delete_collection(COLLECTION)

# 2️⃣ 创建新 collection（维度在这里被“锁定”）
coll = client.get_or_create_collection(
    COLLECTION,
    metadata={"hnsw:space": "cosine"},
)

# 3️⃣ 插入
inserted = skipped = failed = 0
for i, (title, content) in enumerate(sections):
    if not isinstance(content, str):
        skipped += 1
        continue
    if len(content.strip()) < 30 or content.strip().startswith("!["):
        skipped += 1
        continue

    try:
        emb = embedder.get_embedding(content)
        if not emb:
            failed += 1
            continue

        coll.upsert(
            ids=[f"ds_sec_{i}"],
            embeddings=[emb],
            documents=[content],
            metadatas=[{"section_title": title}],
        )
        inserted += 1
    except Exception as e:
        failed += 1
        print("Insert fail:", e)

print(f"Inserted={inserted}, skipped={skipped}, failed={failed}")

# 4️⃣ 最后再把 Knowledge / ChromaDb 指向这个 collection
vector_db = ChromaDb(
    collection=COLLECTION,
    path=CHROMA_PATH,
    embedder=embedder,
    persistent_client=True,
)

knowledge = Knowledge(
    name=COLLECTION,
    vector_db=vector_db,
    max_results=5,
)


Inserted=74, skipped=5, failed=0


In [31]:
q_emb = embedder.get_embedding("什么是过拟合")

res = coll.query(
    query_embeddings=[q_emb],  # 👈 明确告诉 Chroma 用哪个 embedding
    n_results=3,
)

print(res["documents"])


[['### Early stop\n\n用于防止模型**过拟合（一种正则化手段）**。它通过监控模型在validation set上的表现，决定是否需要提前结束训练。\n\n具体流程：设定一个最大迭代次数，记录每次iteration之后的loss，如果loss到某个epoch之后开始出现上升，则继续训练以观察是否是短期波动，如果验证损失在接下来的10个epoch中，都没有比best—loss低，则Early Stopping\n\n### Drop out\n\n 防止模型**过拟合（一种正则化手段）。**\n\n- 在训练过程中，每一层的每个神经元都有drop rate的概率被丢弃（output会直接被设为零），所以这个神经元对该 batch 的训练过程没有贡献。\n- 每次前向传播，都会随机重新丢弃一些神经元，这就是**独立采样Independent Sampling。**导致每个 batch 中的**模型结构都不同**。Dropout 具有类似**ensemble**的效果，从而增强了模型的泛化能力。\n- 在反向传播时，模型只会计算那些**未被丢弃的神经元**的梯度，并根据这些梯度更新权重。\n- **测试阶段**不同于训练阶段，因为在测试时模型要提供稳定的output，因此所有神经元都会被激活。为的是总输出保持与训练阶段一致 To maintain consistency between training and testing outputs，将所有神经元的输出乘以 **(1−dropout rate)。**\n\n好处：防止过拟合，降低对特定神经元的依赖（增强鲁棒性）', '- **person**\n\n![img](https://my.feishu.cn/space/api/box/stream/download/asynccode/?code=NjkwMTg5YzNiYWVjMWUwNTgzMmJhOTc1ODIyMTAxZTJfZ3JOOGtucVJkTzZ5eVhUNzRRYjZyVHRnMWNhTWRDSTNfVG9rZW46VVcyRmJNZ1NDbzhlWk94bmZmWmNSaVFKbnRnXzE3Njk3MTk0MDQ6MTc2OTcyMzAwNF9WNA)\n\n**假设：数据是正态分布。**\n\n取值范围：-1到1，0表示

### 2. State + Tools

- **State**: `used_section_titles`, `round`, `session_start_time`, `interview_ended`
- **Tools**: `get_current_time`, `pick_topic_from_rag`, `end_interview`; Coach is invoked via Team or a tool.

In [41]:
# --- State schema and tools (use run_context for state) ---
from datetime import datetime
from agno.run import RunContext
from agno.tools import tool

# Default session state for interview
INTERVIEW_SESSION_STATE = {
    "used_section_titles": [],
    "round": 0,
    "session_start_time": None,
    "interview_ended": False,
}

@tool
def get_current_time(run_context: RunContext) -> str:
    """Return current time. Use when starting a round or reporting duration."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def pick_topic_from_rag(run_context: RunContext, query: str = "data science interview topic") -> str:
    """Retrieve one topic from the knowledge base that has NOT been used yet. Use this to pick the next question topic. Returns the section content; you should generate a question based on it."""
    used = run_context.session_state.get("used_section_titles") or []
    docs = knowledge.search(query=query, max_results=10)
    for doc in docs:
        title = doc.meta_data.get("section_title") or doc.name
        if title and title not in used:
            run_context.session_state.setdefault("used_section_titles", []).append(title)
            return f"[Section: {title}]\n{doc.content}"
    return "No unused topics left; you may wrap up the interview or repeat from a broader search."

@tool
def end_interview(run_context: RunContext) -> str:
    """Call when the candidate says they want to end or when you decide to finish. Marks interview as ended so summary can be generated."""
    run_context.session_state["interview_ended"] = True
    return "Interview ended. Session summary will be generated."

### 3. Coach Agent (反馈/评分) + request_feedback tool

Coach evaluates candidate answers using RAG-retrieved reference; Interviewer calls Coach via tool when feedback is requested.

In [ ]:
# --- Coach Agent: feedback/scoring; structured output ---
import os
from textwrap import dedent
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from pydantic import BaseModel, Field
from typing import List

# Load environment variables from .env file if it exists
load_dotenv()

# Check if OPENROUTER_API_KEY is set
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment or .env file. Do not commit keys.")

class FeedbackOutput(BaseModel):
    """Structured feedback for one answer."""
    score_1_to_5: int = Field(..., ge=1, le=5, description="Score from 1 to 5")
    key_points_covered: List[str] = Field(..., description="Key points the candidate got right")
    missing_points: List[str] = Field(..., description="Important points missed (from reference)")
    suggestion: str = Field(..., description="One concise suggestion to improve")

coach_agent = Agent(
    name="CoachAgent",
    model=OpenRouter(id="openai/gpt-4o-mini", api_key=os.environ.get("OPENROUTER_API_KEY")),
    instructions=dedent("""
        You are an interview coach. Given the interview question, the candidate's answer, and the reference knowledge,
        provide structured feedback: score (1-5), key points covered, missing points, and one suggestion.
        Be fair and concise. Output in the required structured format.
    """),
    output_schema=FeedbackOutput,
    use_json_mode=True,
    retries=2,
)

In [43]:
# --- request_feedback tool: calls Coach with RAG reference ---
def make_request_feedback_tool(coach_agent, knowledge):
    @tool
    def request_feedback(run_context: RunContext, question: str, candidate_answer: str) -> str:
        """Ask the coach for feedback on the candidate's answer. Provide the question and the candidate's answer. Use after the candidate has answered a question."""
        ref_docs = knowledge.search(query=question, max_results=3)
        reference = "\n\n".join(d.content for d in ref_docs) if ref_docs else "No reference."
        prompt = f"Question: {question}\n\nCandidate answer: {candidate_answer}\n\nReference knowledge:\n{reference}\n\nGive structured feedback (score, key points covered, missing points, suggestion)."
        out = coach_agent.run(prompt)
        if hasattr(out.content, "model_dump"):
            d = out.content.model_dump()
            return f"Score: {d.get('score_1_to_5')}/5 | Covered: {d.get('key_points_covered')} | Missing: {d.get('missing_points')} | Suggestion: {d.get('suggestion')}"
        return str(out.content)

    return request_feedback

request_feedback = make_request_feedback_tool(coach_agent, knowledge)

### 4. Interviewer Agent + Team

Interviewer: 出题/控场, tools + state + RAG. Coach: 反馈/评分. Team = Interviewer + Coach (interviewer can call coach via `request_feedback` tool).

In [ ]:
# --- Interviewer Agent: questions, control, tools, state, RAG ---
import os
from textwrap import dedent
from dotenv import load_dotenv
from agno.agent import Agent
from agno.db.in_memory import InMemoryDb
from agno.models.openrouter import OpenRouter
from pydantic import BaseModel, Field
from typing import List, Optional

# Load environment variables from .env file if it exists
load_dotenv()

# Check if OPENROUTER_API_KEY is set
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment or .env file. Do not commit keys.")

INTERVIEW_TYPE = "technical"
ROLE = "Data Scientist"
SESSION_ID = "interview_full_001"
DB = InMemoryDb()

class InterviewTurn(BaseModel):
    """One interviewer turn: question + type + expected key points, OR session_summary when ending."""
    current_question: Optional[str] = Field(None, description="The interview question to ask the candidate")
    question_type: Optional[str] = Field(None, description="e.g. technical, behavioral")
    expected_key_points: Optional[List[str]] = Field(None, description="Key points to look for in a good answer")
    session_summary: Optional[str] = Field(None, description="If interview ended, summary of the session")

interviewer_agent = Agent(
    name="InterviewAgent",
    model=OpenRouter(id="openai/gpt-4o-mini", api_key=os.environ.get("OPENROUTER_API_KEY")),
    instructions=dedent(f"""
        You are a professional interviewer for a mock data science interview.
        Interview type: {INTERVIEW_TYPE}. Role: {ROLE}.
        Use tools: get_current_time to track rounds; pick_topic_from_rag to get an unused topic and generate ONE question; request_feedback(question, candidate_answer) when the candidate has answered and you want to give feedback; end_interview when the candidate wants to stop or you finish.
        Ask exactly ONE question per turn. Be concise. Output in the required structured format.
        If the candidate says "结束" or "end interview", call end_interview and then output session_summary as a brief recap of the session.
        """),
    tools=[get_current_time, pick_topic_from_rag, end_interview, request_feedback],
    db=DB,
    session_id=SESSION_ID,
    session_state=INTERVIEW_SESSION_STATE,
    add_session_state_to_context=True,
    add_history_to_context=True,
    knowledge=knowledge,
    search_knowledge=True,
    output_schema=InterviewTurn,
    use_json_mode=True,
    retries=3,
)

In [ ]:
# --- Team: Interviewer (lead) + Coach (called via request_feedback tool) ---
import os
from dotenv import load_dotenv
from agno.team import Team
from agno.models.openrouter import OpenRouter

# Load environment variables from .env file if it exists
load_dotenv()

# Check if OPENROUTER_API_KEY is set
if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("Set OPENROUTER_API_KEY in your environment or .env file. Do not commit keys.")

interview_team = Team(
    name="InterviewTeam",
    members=[interviewer_agent, coach_agent],
    model=OpenRouter(id="openai/gpt-4o-mini", api_key=os.environ.get("OPENROUTER_API_KEY")),
)

# Note: Coach is invoked by the interviewer through the request_feedback tool, not by Team routing.
# Team allows both agents to share the same run; the interviewer has the tools and leads the flow.

### 5. Session Summary + Workflow

- **Session Summary**: Triggered when interview ends; generate and display (or include in structured output).
- **Workflow**: Set context → multi-round Q&A (run interviewer until end) → generate summary.

In [49]:
# --- Session Summary: generate when interview_ended (run agent with summary prompt; history in context) ---
def generate_session_summary(agent) -> str:
    """Generate session summary. Call after end_interview. Agent has conversation in context."""
    summary_prompt = "The interview has ended. Based on our conversation, provide a brief session summary (3-5 sentences): topics covered, number of questions, and overall impression. Put the summary in session_summary if you use structured output, or reply with the summary text."
    out = agent.run(summary_prompt)
    if hasattr(out.content, "session_summary") and out.content.session_summary:
        return out.content.session_summary
    if hasattr(out.content, "current_question"):
        return out.content.current_question  # model might put summary in this field
    return str(out.content)

### 6. Workflow: run interview (set context → multi-round → summary)

Single run example: one question + one answer + optional feedback. For full loop, run the next cell repeatedly or use the helper loop below.

In [50]:
# --- Workflow: one round (ask for first question) ---
run = interviewer_agent.run("I'm ready. Please give me the first question (use pick_topic_from_rag to choose an unused topic).")
print("--- Interview Turn ---")
if hasattr(run.content, "model_dump"):
    d = run.content.model_dump()
    for k, v in d.items():
        if v is not None:
            print(f"{k}: {v}")
else:
    print(run.content)

INFO Found 10 documents

--- Interview Turn ---
current_question: Can you explain the concept of overfitting and how to handle it in machine learning? Please include techniques such as data augmentation and how validation might not completely solve the issue.
question_type: technical
expected_key_points: ['Definition of overfitting', 'Impact of overfitting on model performance', 'Data augmentation as a technique to handle overfitting', 'Regularization methods (e.g., L1, L2)', 'Use of validation and potential limitations']


In [51]:
# --- Example: candidate answers, then request feedback (interviewer can call request_feedback tool) ---
# Replace with the actual question from the previous run
last_question = run.content.current_question if hasattr(run.content, "current_question") else "Explain bias and variance."
candidate_answer = "Bias is the difference between model prediction and true value; high bias means underfitting. Variance is sensitivity to data; high variance means overfitting. They trade off."
run2 = interviewer_agent.run(f"I'll answer: {candidate_answer}. Please give me feedback on my answer.")
print("--- Response (may include feedback from Coach) ---")
print(run2.content if not hasattr(run2.content, "model_dump") else run2.content.model_dump())

INFO Found 3 documents

--- Response (may include feedback from Coach) ---
{'current_question': 'Can you explain the concept of overfitting and how to handle it in machine learning? Please include techniques such as data augmentation and how validation might not completely solve the issue.', 'question_type': 'technical', 'expected_key_points': ['Definition of overfitting', 'Impact of overfitting on model performance', 'Data augmentation as a technique to handle overfitting', 'Regularization methods (e.g., L1, L2)', 'Use of validation and potential limitations'], 'session_summary': 'The candidate did not provide a complete answer about handling overfitting. While they correctly explained bias and variance, they failed to mention key techniques such as data augmentation, model selection, and validation limitations.'}


In [52]:
# --- End interview and get Session Summary ---
run_end = interviewer_agent.run("I want to end the interview. Please wrap up.")
print("--- End turn ---")
print(run_end.content.model_dump() if hasattr(run_end.content, "model_dump") else run_end.content)

# Generate and display session summary
summary = generate_session_summary(interviewer_agent)
print("\n--- Session Summary ---")
print(summary)

--- End turn ---
{'current_question': None, 'question_type': None, 'expected_key_points': None, 'session_summary': 'The candidate answered a question about overfitting but did not cover key points around handling overfitting, such as data augmentation and validation limitations. They explained bias and variance but lacked detail on practical techniques.'}

--- Session Summary ---
The interview covered one question related to the concept of overfitting in machine learning. The candidate briefly discussed bias and variance but did not fully address how to handle overfitting or the limitations of validation. Overall, the response indicated a basic understanding of the concepts, but more detailed knowledge of practical techniques was necessary for a stronger performance.


In [ ]:
# --- Optional: Workflow loop (set context → multi-round until "结束" → summary) ---
def run_interview_workflow(agent, max_rounds: int = 5):
    """Run interview: multi-round until user says end or max_rounds. Then print summary."""
    print("Interview started. Say '结束' or 'end' to finish.\n")
    for r in range(max_rounds):
        user_input = input(f"Round {r+1} (you): ").strip()
        if not user_input:
            continue
        if user_input.lower() in ("结束", "end", "结束面试"):
            _ = agent.run("I want to end the interview. Please wrap up.")
            break
        out = agent.run(user_input)
        print("Agent:", getattr(out.content, "current_question", out.content))
    print("\n--- Session Summary ---")
    print(generate_session_summary(agent))

# Uncomment to run interactive loop:
# run_interview_workflow(interviewer_agent, max_rounds=5)